<a href="https://colab.research.google.com/github/LucasAlegre/vote-network/blob/master/vote_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install and import libraries

In [ ]:
! pip install --user graphistry
! pip install python-igraph
! pip install networkx==2.6

In [33]:
import graphistry
import networkx
import pandas as pd
import urllib.request
import numpy as np
from igraph import Graph, plot, summary, read
from itertools import combinations
# Init graphistry
graphistry.register(api=3, protocol="https", server="hub.graphistry.com", username="LucasAlegre", password="")

## Data Retrieval and Processing

In [7]:
start_date = '2019-01-31' #@param {type:"date"}
end_date = '2021-12-30' #@param {type:"date"}
start_date = pd.to_datetime(start_date)
end_date = pd.to_datetime(end_date)

In [25]:
# Collect all votes between start and end dates
for year in range(start_date.year, end_date.year + 1):

    filename = 'votacoesVotos-{}.csv'.format(year)
    urllib.request.urlretrieve('https://dadosabertos.camara.leg.br/arquivos/votacoesVotos/csv/votacoesVotos-{}.csv'.format(year), filename)
    data = pd.read_csv(filename, sep=';')

    # Remove simbolic votes
    data = data[data['voto'] != 'Simbólico']

    # Filter dates
    data['dataHoraVoto'] = pd.to_datetime(data['dataHoraVoto'])    
    mask = (data['dataHoraVoto'] >= start_date) & (data['dataHoraVoto'] <= end_date)
    data = data.loc[mask]
    data.sort_values(by=['dataHoraVoto'], inplace=True)
    
    if year == start_date.year:
        votes = data
    else:
        votes = pd.concat([all_data, data])

In [26]:
#%% Take care of different names for same deputy
for group, df_group in votes.groupby('deputado_id'):
    votes['deputado_nome'].loc[votes['deputado_id'] == group] = sorted(df_group['deputado_nome'].unique())[0]

#%% Partidos que mudaram de nome
votes['deputado_siglaPartido'].replace('PMDB', 'MDB', inplace=True)
votes['deputado_siglaPartido'].replace('PRB', 'REPUBLICANOS', inplace=True)
votes['deputado_siglaPartido'].replace('PR', 'PL', inplace=True)
votes['deputado_siglaPartido'].replace('PATRIOTA', 'PATRI', inplace=True)
votes['deputado_siglaPartido'].replace('PPS', 'CIDADANIA', inplace=True)
#all_data['deputado_siglaPartido'].replace('PPL', np.nan, inplace=True) # PPL foi incorporado
#all_data['deputado_siglaPartido'].replace('PRP', np.nan, inplace=True) # PRP foi incorporado
#all_data['deputado_siglaPartido'].replace('PHS', np.nan, inplace=True) # PHS foi incorporado


# all_data = pd.merge(all_data, motions_themes, on="idVotacao", how="inner") 
#all_data['deputado_siglaPartido'].fillna('S.PART.', inplace=True)
# all_data.groupby('idVotacao')['voto'].count()

#all_data.to_csv('votos_{}_to_{}.csv'.format(start_date, end_date), index=False)

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setit

## Graph Construction

In [31]:
def generalized_similarity(m, min_eps=0.001, max_iter=1000):
    """ Balázs Kovács, "A generalized model of relational similarity," Social Networks, 32(3), July 2010, pp. 197–211
        Based on: https://github.com/dzinoviev/generalizedsimilarity
    """
    arcs0 = m - m.mean(axis=1)[:, np.newaxis]
    arcs1 = m.T - m.mean(axis=0)[:, np.newaxis]

    eps = min_eps + 1
    N = np.eye(m.shape[1])

    iters = 0
    while (eps > min_eps and iters < max_iter) or np.isnan(N).any():
        M = arcs0.dot(N).dot(arcs0.T)
        m = np.sqrt(M.diagonal())
        M = ((M / (m+1e-8)).T / (m+1e-8)).T
        
        Np = arcs1.dot(M).dot(arcs1.T)
        n = np.sqrt(Np.diagonal())
        Np = ((Np / (n+1e-8)).T / (n+1e-8)).T
        eps = np.abs(Np - N).max()
        N = Np

        iters += 1
    return M
    
def filter_edges(edges_list, num_nodes, threshold=None, density=0.1):
    edges, weights = [], []
    if threshold is not None:
        for e in edges_list:
            if e[1] >= threshold:
                edges.append(e[0])
                weights.append(e[1])
    else:
        count = int(num_nodes * (num_nodes - 1) * density / 2)
        edges_list.sort(reverse=True, key=lambda e: e[1])
        edges_list = edges_list[:count]
        edges = [e[0] for e in edges_list]
        weights = [e[1] for e in edges_list]
    return edges, weights

In [29]:
reps = votes['deputado_nome'].unique()
rep_to_ind = {reps[i]: i for i in range(len(reps))}
motions = votes['idVotacao'].unique()
motion_to_ind = {motions[i]: i for i in range(len(motions))}
parties = [p for p in votes['deputado_siglaPartido'].unique() if pd.notna(p)]

vote_matrix = np.zeros((len(reps), len(motions)))
df_grouped = votes.groupby(['idVotacao', 'deputado_nome'])
for group, df_group in df_grouped:
    voto = df_group['voto'].values[0]
    i = rep_to_ind[group[1]]
    j = motion_to_ind[group[0]]
    if voto == "Sim":
        vote_matrix[i,j] = 1
    if voto == "Não":
        vote_matrix[i,j] = -1

M = generalized_similarity(vote_matrix)

In [60]:
edges = []
for dep1, dep2 in combinations(range(len(reps)), 2):
    if M[dep1,dep2] > 0:
        edges.append(((dep1,dep2), M[dep1,dep2]))

#plot_similarity_distribution([e[1] for e in edges if e[1] > 0.99], weight_threshold)

g = Graph(graph_attrs={'name': 'Câmara dos Deputados'}, directed=False)
g.add_vertices(reps)
edges, weights = filter_edges(edges, num_nodes=g.vcount(), threshold=0.99995, density=0.1)
g.add_edges(edges)
g.es['weight'] = weights
# Normalize weights to [0,1]
maxw = max(g.es['weight'])
minw = min(g.es['weight'])
g.es['weight'] = [(e - minw) / (maxw - minw) for e in g.es['weight']]
summary(g)

IGRAPH UNW- 575 37755 -- Câmara dos Deputados
+ attr: name (g), name (v), weight (e)


In [61]:
graph = g.to_networkx()

In [64]:
g = graphistry.bind(source='src', destination='dst', point_title='name')
g.addStyle(bg={'color': 'red'})
g.plot(graph)